<a href="https://colab.research.google.com/github/branley1/smol-course/blob/main/1_instruction_tuning/notebooks/sft_finetuning_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install the requirements in Google Colab
!pip install transformers datasets trl huggingface_hub

# Authenticate to Hugging Face
from huggingface_hub import login
!huggingface-cli login

login()

# for convenience you can create an environment variable containing your hub token as HF_TOKEN


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: fineG

In [2]:
# Import necessary libraries
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, setup_chat_format
import torch

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

# Load the SmolLM2 model and tokenizer
model_smol = AutoModelForCausalLM.from_pretrained("HuggingFaceTB/SmolLM2-135M").to(device)
tokenizer_smol = AutoTokenizer.from_pretrained("HuggingFaceTB/SmolLM2-135M")

# Set up the chat format for SmolLM2
model_smol, tokenizer_smol = setup_chat_format(model=model_smol, tokenizer=tokenizer_smol)


# Load the GPT-2 tokenizer for ds_code_parrot
tokenizer_gpt2 = AutoTokenizer.from_pretrained("gpt2")
if tokenizer_gpt2.pad_token is None:
    tokenizer_gpt2.pad_token = tokenizer_gpt2.eos_token

# Set our name for the finetune to be saved &/ uploaded to
finetune_name = "SmolLM2-FT-MyDataset"
finetune_tags = ["smol-course", "module_1"]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


# Generate with the base model

Here we will try out the base model which does not have a chat template.

In [3]:
# Let's test the base model before training
prompt = "Write a haiku about programming"

# Format with template
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer_smol.apply_chat_template(messages, tokenize=False)

# Generate response
inputs = tokenizer_smol(formatted_prompt, return_tensors="pt").to(device)
outputs = model_smol.generate(**inputs, max_new_tokens=100)
display("Before training:")
display(tokenizer_smol.decode(outputs[0], skip_special_tokens=True))

'Before training:'

'user\nWrite a haiku about programming\nWrite a haiku about programming\nWrite a haiku about programming\nWrite a haiku about programming\nWrite a haiku about programming\nWrite a haiku about programming\nWrite a haiku about programming\nWrite a haiku about programming\nWrite a haiku about programming\nWrite a haiku about programming\nWrite a haiku about programming\nWrite a haiku about programming\nWrite a haiku about programming\nWrite a haiku about programming\nWrite a haiku about programming\nWrite a'

## Dataset Preparation

We will load a sample dataset and format it for training. The dataset should be structured with input-output pairs, where each input is a prompt and the output is the expected response from the model.

**TRL will format input messages based on the model's chat templates.** They need to be represented as a list of dictionaries with the keys: `role` and `content`,.

In [4]:
# Load a sample dataset
from datasets import load_dataset, Dataset

# Tokenizer setup
tokenizer = AutoTokenizer.from_pretrained("gpt2")
if tokenizer.pad_token is None:
  tokenizer.pad_token = tokenizer.eos_token

# Define datasets
# Smoltalk dataset
ds_smoltalk = load_dataset(path="HuggingFaceTB/smoltalk", name="everyday-conversations")

# Process Smoltalk dataset for TRL chat templates
def process_smoltalk(sample):
    # Convert Smoltalk conversation to chat format
    messages = [{"role": msg["role"], "content": msg["content"]} for msg in sample["messages"]]
    return {"messages": messages}

# Apply the processing
ds_smoltalk = ds_smoltalk.map(process_smoltalk, remove_columns=ds_smoltalk["train"].column_names)

# BigCode dataset
ds_big_code = load_dataset(path="bigcode/the-stack-smol-xl", data_dir="data/python")

# Process BigCode dataset for TRL chat templates
def process_big_code(sample):
    # For BigCode, structure the code snippet into a chat format
    messages = [
        {"role": "system", "content": "You are a programming assistant specializing in Python."},
        {"role": "user", "content": "Write or explain the following Python code:"},
        {"role": "assistant", "content": sample["content"]},
    ]
    return {"messages": messages}

# Convert the generator to a list before creating the Dataset
processed_big_code = [process_big_code(sample) for sample in ds_big_code["train"]]
ds_big_code_train = Dataset.from_list(processed_big_code)  # Create a new Dataset object
ds_big_code["train"] = ds_big_code_train  # Replace the original 'train' split with the processed one

# CodeParrot dataset
languages_to_keep = ["Python", "C++", "C", "HTML", "CSS", "SQL", "TypeScript", "TeX"]
ds_code_parrot = load_dataset("codeparrot/github-code", streaming=True, split="train").filter(
    lambda x: x["language"] in languages_to_keep)

# Convert processed CodeParrot datatset into a generator
processed_code_parrot = (process_code_parrot(sample) for sample in ds_code_parrot)

# Process CodeParrot dataset for TRL chat templates
def process_code_parrot(sample):
    messages = [
        {"role": "system", "content": f"You are a programming assistant specializing in {sample['language']}."},
        {"role": "user", "content": "Write or explain the following code snippet:"},
        {"role": "assistant", "content": sample["code"]},
    ]
    return {"messages": messages}

# Map the processing directly on the streaming dataset
processed_code_parrot = ds_code_parrot.map(process_code_parrot)

Using the latest cached version of the dataset since HuggingFaceTB/smoltalk couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'everyday-conversations' at /root/.cache/huggingface/datasets/HuggingFaceTB___smoltalk/everyday-conversations/0.0.0/5a40ecb185e55dd30edf3c24b77e67f6ea0d659b (last modified on Wed Dec 11 01:08:47 2024).


DatasetDict({
    train: Dataset({
        features: ['messages'],
        num_rows: 2260
    })
    test: Dataset({
        features: ['messages'],
        num_rows: 119
    })
})

'ds_smoltalk'

DatasetDict({
    train: Dataset({
        features: ['messages'],
        num_rows: 10000
    })
})

'ds_big_code'

IterableDataset({
    features: Unknown,
    num_shards: 1126
})

'ds_code_parrot'

In [5]:
# General tokenization function
def tokenize_messages_dataset(dataset,tokenizer):
  """
  Tokenizes a dataset using the specified tokenizer.
  Args:
      dataset: The dataset to tokenize.
      tokenizer: The tokenizer to use.
  Returns:
      Tokenized dataset.
  """
  def tokenize_function(sample):
      chat_str = ""
      for msg in sample.get("messages", []):
        if isinstance(msg, dict): # Check if msg is in a dictionary
          role, content = msg.get("role", "").capitalize(), msg.get("content", "")
          chat_str += f"{role}: {content}\n"

      # Tokenize and pad
      return tokenizer(
          chat_str,
          truncation=True,
          padding="max_length",
          max_length=512,
          return_attention_mask=True,
          )

  return dataset.map(tokenize_function, batched=False, remove_columns=["messages"])

# Tokenization function for datasets with 'code'
def tokenize_code_dataset(dataset, tokenizer):
    """
    Tokenizes datasets that have a 'code' field.
    """
    def tokenize_function(sample):
        code_str = sample.get("code", "")
        return tokenizer(
            code_str,
            truncation=True,
            padding="max_length",
            max_length=512,
            return_attention_mask=True
        )

    return dataset.map(
        tokenize_function,
        batched=False, # Process one sample at a time
        remove_columns=["code", "repo_name", "path", "language", "license", "size"]
    )

def add_labels_to_dataset(dataset):
    """
    Adds a 'labels' field to the dataset, which is a copy of 'input_ids'.
    """
    def add_labels(batch):
      # Ensure 'input_ids' is a list before copying
        if isinstance(batch["input_ids"], list):
          batch["labels"] = batch["input_ids"].copy()
        else:
          raise KeyError("'input_ids' should be a list of integers.")
        return batch

    return dataset.map(add_labels)

# Tokenize Smoltalk and BigCode datasets
tokenized_smoltalk = tokenize_messages_dataset(ds_smoltalk, tokenizer_smol)
tokenized_big_code = tokenize_messages_dataset(ds_big_code["train"], tokenizer_smol)

# Tokenize CodeParrot dataset
# Since 'ds_code_parrot' is an IterableDataset, take a subset and convert it to a regular Dataset
tokenized_code_parrot_iterable = tokenize_code_dataset(ds_code_parrot, tokenizer_gpt2).take(512)

# Convert the IterableDataset to a list
tokenized_code_parrot_list = list(tokenized_code_parrot_iterable)

# Create an in-memory Dataset from the list
tokenized_code_parrot_dataset = Dataset.from_list(tokenized_code_parrot_list)

# Add labels to all tokenized datasets
tokenized_smoltalk = add_labels_to_dataset(tokenized_smoltalk)
tokenized_big_code = add_labels_to_dataset(tokenized_big_code)
tokenized_code_parrot = add_labels_to_dataset(tokenized_code_parrot_dataset)

# Inspect the first sample of each tokenized dataset
# print("Smoltalk Sample:", tokenized_smoltalk["train"][0])
# print("BigCode Sample:", tokenized_big_code[0])
# print("CodeParrot Sample:", tokenized_code_parrot_dataset[0])

Map:   0%|          | 0/2260 [00:00<?, ? examples/s]

Map:   0%|          | 0/119 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2260 [00:00<?, ? examples/s]

Map:   0%|          | 0/119 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/512 [00:00<?, ? examples/s]

Smoltalk Sample: {'input_ids': [11126, 42, 21041, 665, 198, 9519, 9531, 42, 38699, 17, 1073, 416, 339, 724, 346, 1834, 47, 198, 11126, 42, 339, 5248, 3012, 327, 253, 10724, 14500, 327, 957, 1867, 17584, 30, 1978, 346, 3730, 634, 2378, 2911, 47, 198, 9519, 9531, 42, 2015, 2378, 10724, 36088, 1453, 48326, 281, 14126, 28, 260, 48148, 898, 28, 284, 260, 44057, 30, 1069, 2316, 1343, 327, 480, 3953, 16351, 284, 13253, 29, 10086, 5656, 30, 198, 11126, 42, 1848, 4598, 1109, 30, 4184, 665, 750, 36088, 281, 260, 11981, 338, 359, 1123, 327, 3168, 47, 198, 9519, 9531, 42, 9230, 28, 260, 25518, 284, 7784, 48096, 10015, 284, 47557, 395, 359, 5412, 4975, 327, 1564, 29, 9263, 36088, 281, 260, 11981, 30, 1069, 2626, 253, 1845, 282, 2123, 284, 32255, 5712, 327, 511, 6399, 30, 198, 11126, 42, 34763, 28, 339, 3060, 1492, 618, 967, 30, 10090, 327, 260, 7400, 17, 198, 9519, 9531, 42, 1206, 2316, 10668, 30, 339, 3826, 346, 1042, 260, 3468, 14500, 327, 469, 17584, 30, 198, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 

In [6]:
# For Smoltalk and BigCode (using SmolLM2)
model_name_smol = "HuggingFaceTB/SmolLM2-135M"
model_smol = AutoModelForCausalLM.from_pretrained(model_name_smol).to(device)
tokenizer_smol = AutoTokenizer.from_pretrained(model_name_smol)
model_smol, tokenizer_smol = setup_chat_format(model=model_smol, tokenizer=tokenizer_smol)

# For CodeParrot (using gpt2)
model_name_gpt2 = "gpt2"  # Or a specific gpt2 variant like "gpt2-medium"
model_gpt2 = AutoModelForCausalLM.from_pretrained(model_name_gpt2).to(device)
tokenizer_gpt2 = AutoTokenizer.from_pretrained(model_name_gpt2)
model_gpt2, tokenizer_gpt2 = setup_chat_format(model=model_gpt2, tokenizer=tokenizer_gpt2)


The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


## Configuring the SFTTrainer

The `SFTTrainer` is configured with various parameters that control the training process. These include the number of training steps, batch size, learning rate, and evaluation strategy. Adjust these parameters based on your specific requirements and computational resources.

In [7]:
!pip install scikit-learn
from sklearn.model_selection import train_test_split

# SFTConfig for SmolLM2
sft_config_smol = SFTConfig(
    output_dir="./sft_output",
    max_steps=2000,  # Adjust based on dataset size and desired training duration
    per_device_train_batch_size=4,  # Set according to your GPU memory capacity
    learning_rate=5e-5,  # Default LR for fine-tuning
    logging_steps=20,  # Log every 20 steps
    save_steps=200,  # Frequency of saving model checkpoints
    evaluation_strategy="steps",  # Evaluate the model at regular intervals
    eval_steps=100,  # Evaluate every 100 steps
    use_mps_device=(
        True if device == "mps" else False
    ),  # Use MPS for mixed precision training
    hub_model_id="finetuned_smol_smolLM2",
)

# SFTTrainer for Smoltalk (SmolLM2)
trainer_smol_smoltalk = SFTTrainer(
    model=model_smol,
    args=sft_config_smol,
    train_dataset=tokenized_smoltalk["train"],
    tokenizer=tokenizer_smol,
    eval_dataset=tokenized_smoltalk.get("test"),
)

# SFTTrainer for BigCode (SmolLM2)
trainer_smol_bigcode = SFTTrainer(
    model=model_smol,
    args=sft_config_smol,
    train_dataset=tokenized_big_code,
    tokenizer=tokenizer_smol,
    eval_dataset=tokenized_big_code,
)

# Configure the SFTTrainer for GPT-2
sft_config_gpt2 = SFTConfig(
    output_dir="./sft_output_codeparrot",
    max_steps=3000, # Increase for larger datasets
    per_device_train_batch_size=4, # Adjust for GPU memory
    learning_rate=5e-5,
    logging_steps=50, # Log every 50 steps
    save_steps=500, # Save checkpoints every 500 steps
    evaluation_strategy="steps", # Evaluate at regular intervals
    eval_steps=100, # Evaluate every 100 steps
    hub_model_id="finetuned_gpt2_codeparrot",
)


# Apply tokenization to the streaming dataset. First, convert it to an in-memory
# dataset using take(N) to fetch N elements, toList(N) converts it to a regular Python list
# Instead of creating a new Dataset object, map the tokenization function directly on the streaming dataset
# and then take a subset of the processed data.
tokenized_code_parrot = list(ds_code_parrot.map(
    lambda x: tokenizer_gpt2(
        x["code"],  # Access the 'code' from the sample
        truncation=True,
        padding="max_length",
        max_length=512,
        return_attention_mask=True
    ),
    batched=True,
    remove_columns=["repo_name", "path", "language", "license", "size"] # Remove all original columns
).take(512)) # Convert to list using list()

tokenized_code_parrot_dataset = Dataset.from_list(tokenized_code_parrot)

# # Split CodeParrot dataset into training and Evaluation Sets (90% train, 10% eval)
split_dataset = tokenized_code_parrot_dataset.train_test_split(test_size=0.1)
train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]

# SFTTrainer for CodeParrot (GPT-2)
trainer_gpt2_codeparrot = SFTTrainer(
    model=model_gpt2,  # Using GPT-2 for CodeParrot
    args=sft_config_gpt2,
    train_dataset=train_dataset,
    tokenizer=tokenizer_gpt2,
    eval_dataset=eval_dataset,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:309: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs
max_steps is given, it will override any value given in num_train_epochs
max_steps is given, it will override any value given in num_train_epochs


## Training the Model

With the trainer configured, we can now proceed to train the model. The training process will involve iterating over the dataset, computing the loss, and updating the model's parameters to minimize this loss.

In [ ]:
import wandb

# 1. Log in to your WandB account
wandb.login(key="4d10e9f814a4d0d285b06e84a8ceed2156ec6ed6")

# wandb.init(project="your_project_name", entity="smol-course")

# Train the models
trainer_smol_smoltalk.train()
trainer_smol_big_code.train()
trainer_gpt2_code_parrot.train()

# Save the models
trainer_smol_smoltalk.save_model(f"./finetuned_smol_smoltalk")
trainer_smol_smoltalk.push_to_hub(
    repo_id="bmmasi1/finetuned_smol_smoltalk",
    tags=["smol-course", "smoltalk", "conversation", "fine-tuned"]
)
trainer_smol_big_code.save_model(f"./finetuned_smol_bigcode")
trainer_smol_bigcode.push_to_hub(
    repo_id="bmmasi1/finetuned_smol_bigcode",
    tags=["smol-course", "bigcode", "code", "fine-tuned"]
)
trainer_gpt2_code_parrot.save_model(f"./finetuned_gpt2_codeparrot")
trainer_gpt2_code_parrot.push_to_hub(
    repo_id="bmmasi1/finetuned_gpt2_codeparrot",
    tags=["smol-course", "codeparrot", "code", "fine-tuned"]
)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Step,Training Loss,Validation Loss


In [ ]:
trainer.push_to_hub(tags=finetune_tags)

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>Bonus Exercise: Generate with fine-tuned model</h2>
    <p>🐕 Use the fine-tuned to model generate a response, just like with the base example..</p>
</div>

In [ ]:
# Test the fine-tuned model on the same prompt

# Let's test the base model before training
prompt = "Write a haiku about programming"

# Format with template
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# Generate response
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)

# TODO: use the fine-tuned to model generate a response, just like with the base example.

## 💐 You're done!

This notebook provided a step-by-step guide to fine-tuning the `HuggingFaceTB/SmolLM2-135M` model using the `SFTTrainer`. By following these steps, you can adapt the model to perform specific tasks more effectively. If you want to carry on working on this course, here are steps you could try out:

- Try this notebook on a harder difficulty
- Review a colleagues PR
- Improve the course material via an Issue or PR.